# The Most in Demand Skills for Data Scientists
## What are employers looking for?
## By Jeff Hale

I'm going to explore which skills and software for Data Scientists are in demand in the US in October 2018 by looking at online job listing sites. I'll be using plotly and pandas to make interactive visualizations for this project.

### See [this Medium article](https://towardsdatascience.com/the-most-in-demand-skills-for-data-scientists-4a4a8db896db) for discussion.


This notebook includes additional analyses not in the Medium article and presents the findings in a slightly different order.

# Please upvote this Kaggle kernel if you find it helpful.

https://stackoverflow.com/questions/46939424/remove-save-and-edit-plot-in-cloud-button-in-plotly-in-offline-mode


In [1]:
# import the usual frameworks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import warnings

from IPython.core.display import display, HTML
from sklearn.preprocessing import MinMaxScaler
    
# import plotly 
import plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls

# for color scales in plotly
import colorlover as cl 

# configure things
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:,.2f}'.format  
pd.options.display.max_columns = 999

py.init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2
%matplotlib inline

List package versions for reproducibility.

In [2]:
#!pip list
# !pwd
# !ls

# Part 1: Data Science Software

Read in the data from the csv. The Google sheet that holds the data is available [here](https://docs.google.com/spreadsheets/d/1lac1H2IgCDCs9LLTQL6yb6MUPN1u4C5fJv_6YjipIaM/edit?usp=sharing).

In [3]:
df = pd.read_csv(
    'File/ds_job_listing_software.csv',
    usecols=['Keyword','LinkedIn', 'Indeed', 'SimplyHired', 'Monster'],
    skiprows=0,
    nrows=37, 
    thousands=',',
    index_col=0,
   
)
df

,LinkedIn,Indeed,SimplyHired,Monster
Keyword,,,,
machine learning,5701,"3,439.00","2,561.00",2340
analysis,5168,"3,500.00","2,668.00",3306
statistics,4893,"2,992.00","2,308.00",2399
computer science,4517,"2,739.00","2,093.00",1900
communication,3404,"2,344.00","1,791.00",2053
mathematics,2605,"1,961.00","1,497.00",1815
visualization,1879,"1,413.00","1,153.00",1207
AI composite,1568,"1,125.00",811.00,687
deep learning,1310,979.00,675.00,606


Cool. We used the read_csv parameters to give us just what we wanted.

## Basic Data Exploration
Let's see what the data look like.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, machine learning to software engineering
Data columns (total 4 columns):
LinkedIn       15 non-null int64
Indeed         14 non-null float64
SimplyHired    14 non-null float64
Monster        15 non-null int64
dtypes: float64(2), int64(2)
memory usage: 600.0+ bytes


In [5]:
df.describe()

,LinkedIn,Indeed,SimplyHired,Monster
count,15.00,14.00,14.00,15.00
mean,"2,337.53","1,657.57","1,264.21","1,269.60"
std,"1,904.66","1,154.49",873.37,958.68
min,413.00,295.00,250.00,200.00
25%,701.50,697.75,525.75,547.00
50%,"1,568.00","1,269.00",982.00,784.00
75%,"3,960.50","2,640.25","2,017.50","1,976.50"
max,"5,701.00","3,500.00","2,668.00","3,306.00"


# Scale and Aggregate 
Scale each column. For each column we'll use MinMaxScaler to subtract the minumum and divide by the original max - original min.

In [6]:
scale = MinMaxScaler()
scaled_df = pd.DataFrame(
    scale.fit_transform(df), 
    columns = df.columns,
    index = df.index)    

In [7]:
scaled_df.head()

,LinkedIn,Indeed,SimplyHired,Monster
Keyword,,,,
machine learning,1.00,0.98,0.96,0.69
analysis,0.90,1.00,1.00,1.00
statistics,0.85,0.84,0.85,0.71
computer science,0.78,0.76,0.76,0.55
communication,0.57,0.64,0.64,0.60


### Scaled Online Job Listings

Let's make a combined score for the large sites by taking the mean of the LinkedIn, Indeed, Monster, and SimplyHired scores.

In [8]:
scaled_df['combined'] = scaled_df[["LinkedIn", "Indeed", "SimplyHired", "Monster"]].mean(axis = 1)
scaled_df.head()

,LinkedIn,Indeed,SimplyHired,Monster,combined
Keyword,,,,,
machine learning,1.00,0.98,0.96,0.69,0.91
analysis,0.90,1.00,1.00,1.00,0.97
statistics,0.85,0.84,0.85,0.71,0.81
computer science,0.78,0.76,0.76,0.55,0.71
communication,0.57,0.64,0.64,0.60,0.61


# Charts with Plotly
Let's make interactive charts with plotly. 

## Let's look at the top 10 by site
Based on the top 10 on LinkedIn, the largest site.

In [9]:
num_sites = 8

y_linkedin = scaled_df.iloc[:num_sites, 0]
y_indeed = scaled_df.iloc[:num_sites, 1]
y_monster = scaled_df.iloc[:num_sites, 2]
y_simply = scaled_df.iloc[:num_sites, 3]

y_linkedin

Keyword
machine learning   1.00
analysis           0.90
statistics         0.85
computer science   0.78
communication      0.57
mathematics        0.41
visualization      0.28
AI composite       0.22
Name: LinkedIn, dtype: float64

In [10]:
indeed = go.Bar(x=scaled_df.index, y=y_indeed, name = "Indeed")
simply = go.Bar(x=scaled_df.index, y=y_simply, name="SimplyHired")
monster = go.Bar(x=scaled_df.index, y=y_monster, name="Monster")
linked = go.Bar(x=scaled_df.index, y=y_linkedin, name="LinkedIn")

data = [linked, indeed, simply, monster]

layout = go.Layout(
    barmode='group',
    title="Top Software in Job Listings for Data Scientists",
    xaxis={'title': 'Software'},
    yaxis={'title': "Scaled Listings", 'separatethousands': True,
    }
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Let's clean things up for publication

In [11]:
p_s_df = scaled_df * 100
p_s_df = p_s_df.round(2)
p_s_df.columns = ['LinkedIn', 'Indeed', 'SimplyHired', 'Monster', 'Score']
p_s_df = p_s_df.sort_values(by=['Score'], ascending = False)
p_s_df.head()

,LinkedIn,Indeed,SimplyHired,Monster,Score
Keyword,,,,,
analysis,89.92,100.00,100.00,100.00,97.48
machine learning,100.00,98.10,95.57,68.90,90.64
statistics,84.72,84.15,85.11,70.80,81.20
computer science,77.61,76.26,76.22,54.73,71.20
communication,56.56,63.93,63.73,59.66,60.97


Let's rename Microsoft Office. Otherwise it gets cut off.

In [12]:
p_s_df.rename(index = {'Microsoft Office': 'MS Office'}, inplace = True)
p_s_df

,LinkedIn,Indeed,SimplyHired,Monster,Score
Keyword,,,,,
analysis,89.92,100.00,100.00,100.00,97.48
machine learning,100.00,98.10,95.57,68.90,90.64
statistics,84.72,84.15,85.11,70.80,81.20
computer science,77.61,76.26,76.22,54.73,71.20
communication,56.56,63.93,63.73,59.66,60.97
mathematics,41.45,51.98,51.57,52.00,49.25
visualization,27.72,34.88,37.34,32.42,33.09
AI composite,21.84,25.90,23.20,15.68,21.65
deep learning,16.96,21.34,17.58,13.07,17.24


## Let's look at the top 20 scores

In [13]:
p_s_df_20 = p_s_df.iloc[:20,:]
p_s_df_20

,LinkedIn,Indeed,SimplyHired,Monster,Score
Keyword,,,,,
analysis,89.92,100.00,100.00,100.00,97.48
machine learning,100.00,98.10,95.57,68.90,90.64
statistics,84.72,84.15,85.11,70.80,81.20
computer science,77.61,76.26,76.22,54.73,71.20
communication,56.56,63.93,63.73,59.66,60.97
mathematics,41.45,51.98,51.57,52.00,49.25
visualization,27.72,34.88,37.34,32.42,33.09
AI composite,21.84,25.90,23.20,15.68,21.65
deep learning,16.96,21.34,17.58,13.07,17.24


## Let's make a bar chart of the top 20 combined scores

In [14]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 20)

data = [
    go.Bar(
        x=p_s_df_20.index,          
        y=p_s_df_20['Score'],
        marker=dict(
            colorscale='Jet',
            color=color_s,
        ),
    )
]

layout = {
    'title': 'Top 20 Technology Skills in Data Scientist Job Listings',
    #'xaxis': {'title': 'Technology'},
    'yaxis': {'title': "Score"}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Let's make a bar chart of all the combined scores

In [15]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 44)

data = [
    go.Bar(
        x=p_s_df.index,          
        y=p_s_df['Score'],
        marker=dict(
            colorscale='Jet',
            #cauto=True,
            color=color_s,
        ),
        # text=p_s_df['Score'],
        # textposition='outside',
        # textfont=dict(size=10)
    )
]

layout = {
    'title': 'Technology Skills in Data Scientist Job Listings',
    #'xaxis': {'tickmode': 'linear'},
    'yaxis': {'title': "Score"}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## % of job listings with software skills


Total "data scientist" listings.

In [16]:
total_ds = {
    'LinkedIn': 8610,
    'Indeed': 5138,
    'SimplyHired': 3829,
    'Monster': 3746,
}

In [17]:
for key, value in total_ds.items():
    df[key + " %"] = df[key] / value * 100
df

,LinkedIn,Indeed,SimplyHired,Monster,LinkedIn %,SimplyHired %,Indeed %,Monster %
Keyword,,,,,,,,
machine learning,5701,"3,439.00","2,561.00",2340,66.21,66.88,66.93,62.47
analysis,5168,"3,500.00","2,668.00",3306,60.02,69.68,68.12,88.25
statistics,4893,"2,992.00","2,308.00",2399,56.83,60.28,58.23,64.04
computer science,4517,"2,739.00","2,093.00",1900,52.46,54.66,53.31,50.72
communication,3404,"2,344.00","1,791.00",2053,39.54,46.77,45.62,54.81
mathematics,2605,"1,961.00","1,497.00",1815,30.26,39.10,38.17,48.45
visualization,1879,"1,413.00","1,153.00",1207,21.82,30.11,27.50,32.22
AI composite,1568,"1,125.00",811.00,687,18.21,21.18,21.90,18.34
deep learning,1310,979.00,675.00,606,15.21,17.63,19.05,16.18


Let's make an average of those % columns.

In [18]:
df['avg_percent'] = df.iloc[:,-4:].mean(axis=1)
df

,LinkedIn,Indeed,SimplyHired,Monster,LinkedIn %,SimplyHired %,Indeed %,Monster %,avg_percent
Keyword,,,,,,,,,
machine learning,5701,"3,439.00","2,561.00",2340,66.21,66.88,66.93,62.47,65.62
analysis,5168,"3,500.00","2,668.00",3306,60.02,69.68,68.12,88.25,71.52
statistics,4893,"2,992.00","2,308.00",2399,56.83,60.28,58.23,64.04,59.85
computer science,4517,"2,739.00","2,093.00",1900,52.46,54.66,53.31,50.72,52.79
communication,3404,"2,344.00","1,791.00",2053,39.54,46.77,45.62,54.81,46.68
mathematics,2605,"1,961.00","1,497.00",1815,30.26,39.10,38.17,48.45,38.99
visualization,1879,"1,413.00","1,153.00",1207,21.82,30.11,27.50,32.22,27.91
AI composite,1568,"1,125.00",811.00,687,18.21,21.18,21.90,18.34,19.91
deep learning,1310,979.00,675.00,606,15.21,17.63,19.05,16.18,17.02


In [19]:
df = df.sort_values(by="avg_percent", ascending = False)
df

,LinkedIn,Indeed,SimplyHired,Monster,LinkedIn %,SimplyHired %,Indeed %,Monster %,avg_percent
Keyword,,,,,,,,,
analysis,5168,"3,500.00","2,668.00",3306,60.02,69.68,68.12,88.25,71.52
machine learning,5701,"3,439.00","2,561.00",2340,66.21,66.88,66.93,62.47,65.62
statistics,4893,"2,992.00","2,308.00",2399,56.83,60.28,58.23,64.04,59.85
computer science,4517,"2,739.00","2,093.00",1900,52.46,54.66,53.31,50.72,52.79
communication,3404,"2,344.00","1,791.00",2053,39.54,46.77,45.62,54.81,46.68
mathematics,2605,"1,961.00","1,497.00",1815,30.26,39.10,38.17,48.45,38.99
visualization,1879,"1,413.00","1,153.00",1207,21.82,30.11,27.50,32.22,27.91
AI composite,1568,"1,125.00",811.00,687,18.21,21.18,21.90,18.34,19.91
deep learning,1310,979.00,675.00,606,15.21,17.63,19.05,16.18,17.02


## Let's chart the full list and then the top 20.

In [27]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 44)

data = [
    go.Bar(
        x=df.index,          
        y=df['avg_percent'],
        marker=dict(
            colorscale='Jet',
            color=color_s,
        ),
    )
]

layout = dict(
    title="Technology Skills in Data Scientist Job Listings",
    yaxis=dict(title="% of Listings",)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, show_link=False)

In [21]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 20)

data = [
    go.Bar(
        x=df.index,          
        y=df.iloc[:20, -1],
        marker=dict(
            colorscale='Jet',
            color=color_s,
        ),
    )
]

layout = dict(
    title="Top 20 Technology Skills in Data Scientist Job Listings",
    yaxis=dict(title="% of Listings",)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Glassdoor comparision

Glassdoor has some similar analysis from early 2017 [here](https://www.glassdoor.com/research/data-scientist-personas/). See [my larger Medium article](https://towardsdatascience.com/the-most-in-demand-skills-for-data-scientists-4a4a8db896db) for discussion.

In [22]:
# percentage of data scientist jobs on glassdoor early 2017
glassdoor = {
    'Python': 72,
    'R': 64,
    'SQL': 51,
    'Spark': 27,
    'Hadoop': 39,
    'Java': 33,
    'SAS': 30,
    'Tableau': 14,
    'Hive': 17,
    'Matlab': 20
}

In [23]:
# make a data frame of just these
# could just merge the series and df 

series_gd = pd.Series(glassdoor)
series_gd

Hadoop     39
Hive       17
Java       33
Matlab     20
Python     72
R          64
SAS        30
SQL        51
Spark      27
Tableau    14
dtype: int64

In [24]:
df_gd = pd.merge(
    df, 
    pd.DataFrame(
        series_gd, 
        columns=['gd_percent']), 
    right_index=True, 
    left_index=True, 
    how="inner"
)

# df_gd = df_gd[""]
# df_gd.columns = 
df_gd

,LinkedIn,Indeed,SimplyHired,Monster,LinkedIn %,SimplyHired %,Indeed %,Monster %,avg_percent,gd_percent


In [25]:
avg = go.Bar(x=df_gd.index, y=df_gd['avg_percent'], name="Oct. 2018 Averaged")
gd = go.Bar(x=df_gd.index, y=df_gd['gd_percent'], name="Early 2017 Glassdoor")


data = [gd, avg]

layout = go.Layout(
    barmode='group',
    title="Technology Skills 2017 Glassdoor Compared to 2018 Average",
    xaxis={'title': 'Technology'},
    yaxis={'title': '% of Listings'},
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

# Part 2: General Data Science Skills
Let's look at general data science skills next.

In [26]:
df_skills = pd.read_csv(
    '../input/data-scientist-general-skills-2018-revised/ds_general_skills_revised.csv',
    nrows=15,
    thousands=',',
    index_col=0,  
    )
df_skills

# this dataset was updated with "data engineering included 10/15/18"

FileNotFoundError: File b'../input/data-scientist-general-skills-2018-revised/ds_general_skills_revised.csv' does not exist

All terms were searched with *"data scientist"* again. The skills are tricky because there is some overlap. 

I combined *AI* and *artificial intelligence* because one is the abbreviation of the other. I did the same for *NLP* and *natural language processing*. I combined these terms be adding the totals together and subtracting the number of listings that had both terms in them.

Let's rename the composite indices.

In [ ]:
df_skills.rename(index={'AI composite': 'AI', 'NLP composite': 'NLP'}, inplace = True)
df_skills

# Scale and Aggregate 
Scale each column. For each column we'll use MinMaxScaler to subtract the minumum and divide by the original max - original min.

In [ ]:
scale = MinMaxScaler()
scaled_df = pd.DataFrame(
    scale.fit_transform(df_skills), 
    columns = df_skills.columns,
    index = df_skills.index)    

In [ ]:
scaled_df

### Scaled Online Job Listings

Let's make a combined score for the large sites by taking the mean of the LinkedIn, Indeed, and SimplyHired scores.

In [ ]:
scaled_df['big_sites'] = scaled_df[["LinkedIn", "Indeed", "SimplyHired", "Monster"]].mean(axis = 1)
scaled_df.sort_values(by = 'big_sites', ascending = False)

## Let's look at them by site


In [ ]:
y_linkedin = scaled_df.iloc[:, 0]
y_indeed = scaled_df.iloc[:, 1]
y_simply = scaled_df.iloc[:, 2]
y_monster = scaled_df.iloc[:, 3]

y_linkedin

In [ ]:
indeed = go.Bar(x=scaled_df.index, y=y_indeed, name = "Indeed")
simply = go.Bar(x=scaled_df.index, y=y_simply, name="SimplyHired")
linked = go.Bar(x=scaled_df.index, y=y_linkedin, name="LinkedIn")
monster = go.Bar(x=scaled_df.index, y=y_monster, name="Monster")

data = [linked, indeed, simply, monster]

layout = go.Layout(
    barmode='group',
    title="Data Science Skills in Online Job Listings",
    xaxis={'title': 'Skills'},
    yaxis={'title': "Scaled Listings", 'separatethousands': True,
    }
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Let's clean things up for publication

In [ ]:
p_s_df = scaled_df * 100
p_s_df = p_s_df.round(2)
p_s_df.columns = ['LinkedIn', 'Indeed', 'SimplyHired', 'Monster', 'Score']
p_s_df = p_s_df.sort_values(by=['Score'], ascending = False)
p_s_df

## Let's make a bar chart of the combined scores

In [ ]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 14)

data = [
    go.Bar(
        x=p_s_df.index,          
        y=p_s_df['Score'],
        marker=dict(
            colorscale='Jet',
            #cauto=True,
            color=color_s,
        ),
        # text=p_s_df['Score'],
        # textposition='outside',
        # textfont=dict(size=10)
    )
]

layout = {
    'title': 'General Skills in Data Scientist Job Listings',
    'xaxis': {'tickmode': 'linear'},
    'yaxis': {'title': "Score"}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Let's look at these general skills by % of occurrence, too.

In [ ]:
total_ds = {
    'LinkedIn': 8610,
    'Indeed': 5138,
    'SimplyHired': 3829,
    'Monster': 3746,
}

In [ ]:
for key, value in total_ds.items():
    df_skills[key + " %"] = df_skills[key] / value * 100
df_skills

Let's make an average of those % columns.

In [ ]:
df_skills['avg_percent'] = df_skills.iloc[:,-4:].mean(axis=1)
df_skills

In [ ]:
df_skills = df_skills.sort_values(by="avg_percent", ascending = False)
df_skills

## Let's chart the full list.

In [ ]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 16)

data = [
    go.Bar(
        x=df_skills.index,          
        y=df_skills['avg_percent'],
        marker=dict(
            colorscale='Jet',
            color=color_s,
        ),
    )
]

layout = dict(
    title="General Skills in Data Scientist Job Listings",
    yaxis=dict(title="Average % of Listings",)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

# A few summary stats

In [ ]:
ds_results = pd.Series({
    'LinkedIn': 8610,
    'Indeed': 5138, 
    'SimplyHired': 3829,
    'Monster': 3746,
    'AngelList': 658
})
ds_results

In [ ]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 5)

data = [
    go.Bar(
        x=ds_results.index,          
        y=ds_results.values,
        marker=dict(
            colorscale='Jet',
            #cauto=True,
            color=color_s,
        ),
        # text=p_s_df['Score'],
        # textposition='outside',
        # textfont=dict(size=10)
    )
]

layout = {
    'title': "Data Scientist Job Listings",
    'xaxis': {'title': 'Website'},
    'yaxis': {'title': 'Listings', 'separatethousands': True,}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)



# Pleave upvote if you found this interesting or informative!

## Postscript: Configuring Plotly

Plotly is super cool, but setup can be a bit tricky and there is a lot of old documentation mixed in with relevant documentation. Here's how to get up and running.

Follow the instructions on [plotly's site](https://github.com/plotly/plotly.py).

To install on Mac/Linux from the command line with support for plotly's magic, depending on what you already have installed see the plotly docs. Here's what I had to run with a relatively new conda install:

conda install -c plotly plotly=3.3.0 <br>
conda install -c conda-forge nodejs <br>
conda install jupyterlab=0.34 "ipywidgets>=7.2" <br>
export NODE_OPTIONS=--max-old-space-size=4096 <br>
jupyter labextension install @jupyter-widgets/jupyterlab-manager@0.37 --no-build <br>
jupyter labextension install plotlywidget@0.4.0 --no-build <br>
jupyter labextension install @jupyterlab/plotly-extension@0.17.2 --no-build <br>
jupyter lab build <br>
unset NODE_OPTIONS <br>